# Grading students' submissions

## Overall solution
The CLI interface for the autograder works only with docker containers.  
If we want to grade without containers, we will need to use other means.

In this notebook, we use the `grade_submission` API.  
But is is very limited:
* It grades single submissions, not batches => we need to automate for batch + use parallel threads for performance?
* It returns an object, that we will need to "parse" to reconstruct a moodle grading sheet (at least the CLI interface generates a CSV file... maybe it is possible to reuse part of the code?)

TODO: for the moment I tested grading on individual .ipynb files, need to test on zip files

## How to use it

* Go into the dist folder, edit the student version of the assignment and use the public tests
* Then try to grade it with the code below

In [1]:
from otter.api import grade_submission
import glob # to be able to refer to the autograder zip without having to check its name (changes each time it is generated)

In [3]:
graderzip = glob.glob("dist/autograder/assignment-autograder_*.zip")[0]
graderzip

'dist/autograder/assignment-autograder_2022_10_21T12_57_58_304477.zip'

In [4]:
resultsobject = grade_submission("dist/student/assignment.ipynb", graderzip, quiet=True)
resultsobject

e1 results:
    e1 - 1 message: These tests indicate that your function does not compute the square of the number. What operator did you choose? Did you test your function on different values?

    e1 - 1 result:
        Trying:
            assert square(1) == 1
        Expecting nothing
        **********************************************************************
        Line 1, in e1 0
        Failed example:
            assert square(1) == 1
        Exception raised:
            Traceback (most recent call last):
              File "/usr/lib/python3.8/doctest.py", line 1336, in __run
                exec(compile(example.source, filename, "single",
              File "<doctest e1 0[0]>", line 1, in <module>
                assert square(1) == 1
            AssertionError
        Trying:
            assert square(0) == 0
        Expecting nothing
        **********************************************************************
        Line 2, in e1 0
        Failed example:
            

In [5]:
resultsobject.to_dict()

{'e1': {'score': 0,
  'possible': 1.0,
  'name': 'e1',
  'path': 'tests/e1.py',
  'test_cases': [{'name': 'e1 - 1',
    'body': '>>> assert square(1) == 1\n>>> assert square(0) == 0\n',
    'hidden': False,
    'points': 0.0,
    'success_message': 'Well done! However, keep in mind that these tests are not complete and do not guarantee your code is correct. Did you test your function on different values?',
    'failure_message': 'These tests indicate that your function does not compute the square of the number. What operator did you choose? Did you test your function on different values?'},
   {'name': 'e1 - 2',
    'body': '>>> assert square(2.5) == 6.25\n>>> assert square(-1) == 1\n',
    'hidden': True,
    'points': 1,
    'success_message': 'It looks like your square function indeed computes the square of the number, well done!',
    'failure_message': 'It is probable that your function computes a different power than 2. Did you test your function with different values?'}],
  'all